In [14]:
from typing import DefaultDict
from numpy import left_shift
import csv
import pandas as pd
from copy import deepcopy
global_map={}
def read_csv_file(file_name):
    with open(file_name, newline='', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        data = [dict((k.strip(), v) for k, v in row.items()) for row in reader]
    return data

def parse_functional_dependencies():
    fds = []
    while True:
        fd_input = input("Input Functional Dependencies (or 'exit'): ")
        if fd_input == "exit":
            break
        fd = fd_input.split('->')
        fds.append((set(x.strip() for x in fd[0].split(',')), set(x.strip() for x in fd[1].split(','))))
    return fds

def decompose_to_1NF(fds,data,multi_value_attributes):



  original_df = pd.DataFrame(data)
  #print(original_df)

  for attr in multi_value_attributes:
    # Split the "Courses" column into a list of courses
    original_df[attr] = original_df[attr].str.split(',')

    # Explode the "Courses" column to create multiple rows for each course
    result_df = original_df.explode(attr).reset_index(drop=True)

    original_df = result_df


  #print("Resulting DataFrame:")
  #print(result_df)
  data = result_df.to_dict(orient='records')
  return data
  #print(data)



def find_normal_form(data, fds, key,target_nf_choice, mvds):
    #to-do
    #target_nf = 3
    key = set(key)
    all_attributes = set(data[0].keys())

    table_fds={}
    tables_of_2nf_dfs={}
    # Check for 1NF
    multi_value_attributes = is_in_1NF(data)
    if multi_value_attributes == set():
      print("in 1NF")
      print("checking for 2NF...")
    else:
      print("Not in 1NF")
      #print("multi_value_attributes : {}",multi_value_attributes)
      data = decompose_to_1NF(fds,data,multi_value_attributes)
      print("printing table after decomposition...")
      print(data)

    if target_nf_choice >=  '2':
      # Check for 2NF
      violated_fds,correct_fds = is_in_2NF(fds,key)
      #print("=========violated==========")
      #print(violated_fds)
      #print("=========correct_fds==========")
      #print(correct_fds)
      if violated_fds == []:
        print("Table is in 2NF")
        table_fds = {}
        tables_of_2nf_dfs = {}
        #print("============== fds ==========")
        #print(fds)
        #print("============== key ==========")
        #print(key)
        table_fds["_".join(key)] = fds
        tables_of_2nf_dfs["_".join(key)] = converDataToDataFrame(data)
        populateGlobalMap(fds,key,data,violated_fds,correct_fds)
        if target_nf_choice >=  '3':
          table_fds,tables_of_2nf_dfs = is_in_3NF(table_fds,tables_of_2nf_dfs,key)
      else:
        print("Table is not in 2NF")
        table_fds,tables_of_2nf_dfs = decompose_to_2NF(data,key,violated_fds,correct_fds,fds)
        for k,v in tables_of_2nf_dfs.items():
          print("Primary Key : ",k)
          print(v)
        if target_nf_choice >=  '3':
          table_fds,tables_of_2nf_dfs = is_in_3NF(table_fds,tables_of_2nf_dfs,key)
          for k,v in tables_of_2nf_dfs.items():
            print("Primary Key : ",k)
            print(v)
    if(target_nf_choice ==  'B'):
      #print("======= once more =======")
      #print(table_fds)
      #print("======= once more =======")
      #print(tables_of_2nf_dfs)
      #print("choice is B")
      problematic_fds = is_in_BCNF(table_fds, tables_of_2nf_dfs,key)
      if problematic_fds == {}:
        print("============== Table is in BCNF ===========")
        #print(tables_of_2nf_dfs.values())
      else:
        #print("==== problematic_fds =======")
        #print(problematic_fds)
        print("============== Table is not in BCNF ===========")
        table_fds, tables_of_2nf_dfs = decompose_to_BCNF(problematic_fds,table_fds, tables_of_2nf_dfs,key)
        for k,v in tables_of_2nf_dfs.items():
          print("Primary Key : ",k)
          print(v)
        #print("================after resolving ======")
        #print(table_fds)
        #print(tables_of_2nf_dfs)
    if(target_nf_choice ==  '4'):
      is_in_4NF(table_fds,tables_of_2nf_dfs,mvds,key)


def is_in_4NF(table_fds,tables_of_2nf_dfs,mvds,key):
      #converting mvd into map of  {k|v1,v2}
      mvd_map = {}
      for each_mvd in mvds:
        if next(iter(each_mvd[0])) not in mvd_map:
          mvd_map[next(iter(each_mvd[0]))] = []
        mvd_map[next(iter(each_mvd[0]))].append(next(iter(each_mvd[1])))

      #print("mvd_map : ",mvd_map)
      flag = 0
      for table_name,df in tables_of_2nf_dfs.items():
        table_attrs = df.keys()
        table_mvds={}
        for mvd in mvds:
          mvd_attrs = mvd[1].union(mvd[0])
          if mvd_attrs.issubset(table_attrs):
            table_mvds.setdefault(table_name,[]).append(mvd)
        #print(table_mvds)
        B,C = mvd_map[table_name]
        if B not in key and C not in key:
          print("Table is not in 4NF")
          flag=1
          orig_table = tables_of_2nf_dfs[table_name]
          print("========Tables=========")
          cols = []
          cols.append(table_name)
          cols.append(B)
          print(orig_table[cols])
          cols = []
          cols.append(table_name)
          cols.append(C)
          print(orig_table[cols])

      if flag == 0:
        print("Table is in 4NF")

        #mvd_map :  {'Ename': ['Pname', 'Dname']}
        #{'Ename': [({'Ename'}, {'Pname'}), ({'Ename'}, {'Dname'})]}





def decompose_4nf(data, fds, mvds):
    # Implementing basic 4NF decomposition using given MVDs.
    all_attrs = set(data[0].keys())
    decompositions = []
    for mvd in mvds:
        left, right = mvd
        decompositions.append(left.union(right))
        all_attrs -= right

    decompositions.append(all_attrs)
    return decompositions

def decompose_to_5nf(data, fds):
    # Identify join dependencies and relevant attributes
    join_dependencies, relevant_attributes = identify_join_dependencies(fds)

    # Perform lossless-join decomposition based on identified JDs
    decomposed_tables = lossless_join_decomposition(data, join_dependencies, relevant_attributes)

    return decomposed_tables

def identify_join_dependencies(fds):
    join_dependencies = []  # List of identified join dependencies
    relevant_attributes = set()  # Set of relevant attributes

    # Create a dictionary to store the attributes that are functionally dependent on each key
    fd_dict = {}

    for fd in fds:
        left, right = fd
        # Check if the right-hand side is a subset of the key (i.e., it determines all key attributes)
        if left.issubset(right):
            join_dependencies.append(fd)
            relevant_attributes.update(right)
        else:
            # Update the dictionary with attributes that are functionally dependent on the left-hand side
            for attr in left:
                if attr in fd_dict:
                    fd_dict[attr].update(right)
                else:
                    fd_dict[attr] = set(right)

    # Find additional join dependencies using the dictionary
    for attr, dependent_attrs in fd_dict.items():
        if attr in relevant_attributes:
            for dep_attr in dependent_attrs:
                if dep_attr != attr:
                    # If dep_attr is not already in join_dependencies, add it as a join dependency
                    if not any(dep_fd == (set([attr]), set([dep_attr])) for dep_fd in join_dependencies):
                        join_dependencies.append((set([attr]), set([dep_attr])))

    return join_dependencies, relevant_attributes


def lossless_join_decomposition(data, join_dependencies, relevant_attributes):
    decomposed_tables = []

    # Create a set of attributes that have not been covered by any decomposition yet
    uncovered_attributes = set(data[0].keys())

    while uncovered_attributes:
        # Choose a set of attributes to create a new decomposed table
        chosen_attributes = choose_attributes(uncovered_attributes, relevant_attributes)
        decomposed_tables.append(chosen_attributes)
        uncovered_attributes -= chosen_attributes

    return decomposed_tables

def choose_attributes(uncovered_attributes, relevant_attributes):
    chosen_attributes = set()

    # Iterate through the relevant attributes to find a suitable set of attributes
    for attr in relevant_attributes:
        if attr in uncovered_attributes:
            chosen_attributes.add(attr)
            break

    # If no relevant attribute is left to choose, pick any uncovered attribute
    if not chosen_attributes:
        chosen_attributes.add(uncovered_attributes.pop())

    return chosen_attributes


# Example of usage:
# Define your functional dependencies (FDs)
#fds = [({'A'}, {'B'}), ({'B', 'C'}, {'D', 'E'}), ({'D'}, {'F'}), ...]









def decompose_to_BCNF(problematic_fds,table_fds, tables_of_2nf_dfs,key):
  print("================  decompose_to_BCNF ============")
  #print(problematic_fds)
  #print(table_fds)
  #print(tables_of_2nf_dfs)
  #print(key)

  for problematic_table_name,problematic_fd_indices in problematic_fds.items():
    orig_df = tables_of_2nf_dfs[problematic_table_name]
    for ind in problematic_fd_indices:
      each_problematic_fd = table_fds[problematic_table_name][ind]


      X = each_problematic_fd[0]
      Y = each_problematic_fd[1]

      #resolve
      #include new table
      l_key = '_'.join(X)
      if l_key not in table_fds:
        table_fds[l_key] = Y.union(X)

      #include new tables' df
      if l_key not in tables_of_2nf_dfs:
        tables_of_2nf_dfs[l_key] = orig_df[list(table_fds[l_key])]

    #remove p_fd from original table
    problematic_fd_indices.sort(reverse=True)
    for ind in problematic_fd_indices:
      table_fds[problematic_table_name].pop(ind)

  return table_fds, tables_of_2nf_dfs





def is_in_BCNF(table_fds,tables_of_2nf_dfs, candidate_key):
  problematic_fds = {}
  for table_name, fds in table_fds.items():
    # Initialize an empty set for candidate key
    for i, fd in enumerate(fds):
      left_hand_side = fd[0]
      # Check if the left-hand side of the FD is not a superkey
      if not is_superkey(candidate_key, fds[:i]):
        problematic_fds.setdefault(table_name, []).append(i)
      candidate_key.update(left_hand_side)
  return problematic_fds

def is_superkey(candidate_key, fds):
    # Check if candidate_key is a superkey by ensuring it determines all attributes in the table
    for fd in fds:
        if not candidate_key.issuperset(fd[1]):
            return False
    return True


def populateGlobalMap(fds,primary_key_set,data,violated_fds,correct_fds):
  global global_map
  # Include the main table with the primary key and remaining attributes
  map = {}
  for fd in fds:
  #fd[0] -> left
  #fd[1] -> right
   if fd[0].issubset(primary_key_set) and not fd[0] == primary_key_set:
     map["_".join(list(fd[0]))]=None
     #and
   elif fd[0] == primary_key_set:
     map["_".join(list(fd[0]))]=None
     #tables_of_2nf["_".join(list(fd[0]))]=set()


  for vfd in violated_fds:
      for every_val_in_rhs in vfd[1]:
        if every_val_in_rhs not in map:
          map[every_val_in_rhs]="_".join(vfd[0])

  for cfd in correct_fds:
    for every_val_in_rhs in cfd[1]:
      map[every_val_in_rhs]="_".join(cfd[0])

  for vfd in violated_fds:
      if '_'.join(vfd[0]) not in map:
        map['_'.join(vfd[0])]=None
      non_primes_in_lhs = []
      for attr in vfd[0]:
        if attr not in primary_key_set:
          non_primes_in_lhs.append(attr)
      for ever_nonprime_attr in non_primes_in_lhs:
        map[ever_nonprime_attr] = "_".join(vfd[0])
  #print("========== map ===========")
  #print(map)

  global_map = map

def converDataToDataFrame(data):
  original_df = pd.DataFrame(data)
  return original_df
def is_in_3NF(table_fds,tables_of_2nf_dfs,key):
  #print("============ tables_of_2nf_dfs ========")
  #print(tables_of_2nf_dfs)
  #{'StudentID': [({'StudentID'}, {'LastName', 'FirstName'})], 'Course': [({'Course'}, {'CourseEnd', 'Professor', 'CourseStart'}), ({'Professor'}, {'ProfessorEmail'})]}
  faulty_dependency_indexes = {}
  temp_table_fds = deepcopy(table_fds)
  for table_name,list_of_fds in table_fds.items():
    if list_of_fds == []:
      continue
    attr_list = set()
    for fd in list_of_fds:
      attr_list.update(fd[1].union(fd[0]))

    transitive_dependencies,faulty_dependency_indexes = is_in_3NF_internal(list_of_fds,attr_list ,table_name, faulty_dependency_indexes,key)
    #print("======== transitive_dependencies=========")
    #print(transitive_dependencies)
    if transitive_dependencies == list():
      print(f"table {table_name} is in 3NF")
    else:
      print(f"table {table_name} is not in 3NF")
    #print("======== faulty_dependency_indexes=========")
    #print(faulty_dependency_indexes)
    for key,val in faulty_dependency_indexes.items():
      faulty_dependency_indexes[key]=list(set(val))
    #print("========== working till this point ================")
    decompose_to_3NF(transitive_dependencies,temp_table_fds, tables_of_2nf_dfs,faulty_dependency_indexes)
  table_fds = temp_table_fds
  #print("======== final table_fds =========")
  #print(table_fds)
  #print("======== final tables_of_2nf_dfs hii =========")
  #print(tables_of_2nf_dfs)
  return table_fds,tables_of_2nf_dfs





def decompose_to_3NF(transitive_dependencies,temp_table_fds, tables_of_2nf_dfs,faulty_dependency_indexes):


  for faulty_table,faulty_fd_indexes in faulty_dependency_indexes.items():
    data_df = tables_of_2nf_dfs[faulty_table]
    for each_faulty_dependency_index in faulty_fd_indexes:
      faulty_fd = temp_table_fds[faulty_table][each_faulty_dependency_index]
      #print("faulty_fd : ",faulty_fd)

      #take right side attributes of faulty_fd
      removable_cols = list(faulty_fd[1])

      #create new tble faulty_fd[1].unionfaulty_fd[0] with faulty_fd[0] as another pk
      table_attrs = faulty_fd[1].union(faulty_fd[0])

      #print("table_attrs : ",table_attrs)
      #print(data_df)
      new_df = data_df[list(table_attrs)]
      #print(new_df)

      tables_of_2nf_dfs["_".join(faulty_fd[0])] = new_df

      #update temp_table_fds with new pk key and new fd list
      temp_table_fds.setdefault("_".join(faulty_fd[0]), []).append(faulty_fd)


      #trim from data
      data_df = data_df.drop(columns=removable_cols)
      #print(data_df)

      #update tables_of_2nf_dfs with new pk key and new df table
      tables_of_2nf_dfs[faulty_table] = data_df


      #print(tables_of_2nf_dfs)

      #print(" ==== before popping =======")
      #print(temp_table_fds[faulty_table])
      temp_table_fds[faulty_table].pop(each_faulty_dependency_index)
      #print(temp_table_fds[faulty_table])




    #print("====================================")
    #print(temp_table_fds)
    #print("====================================")
    #print(tables_of_2nf_dfs)








def decompose_to_2NF(data, primary_key,violated_fds,correct_fds,fds):
    global global_map
    decomposed_relations = []
    tables_of_2nf = {}
    table_fds = {}

    primary_key_set = set(primary_key)
    tables_of_2nf["_".join(primary_key_set)]=primary_key_set
    table_fds["_".join(primary_key_set)] = []
    #========= violated_fds =========
    #[({'Ssn'}, {'Ename'}), ({'Pnumber'}, {'Pname'}), ({'Pnumber'}, {'Plocation'})]
    for fd in violated_fds:
      # Create a new table
      table_attrs = fd[1].union(fd[0])
      temp_key = "_".join(fd[0])
      if temp_key not in tables_of_2nf:
        tables_of_2nf[temp_key] = set()
      tables_of_2nf[temp_key].update(table_attrs)

      if temp_key not in table_fds:
        table_fds[temp_key] = list()
      table_fds[temp_key].append(fd)



    #print("******* tables_of_2nf ******")
    #print(tables_of_2nf)


    # Include the main table with the primary key and remaining attributes
    map = {}
    for fd in fds:
      #fd[0] -> left
      #fd[1] -> right
      if fd[0].issubset(primary_key_set) and not fd[0] == primary_key_set:
        map["_".join(list(fd[0]))]=None
        #and
      elif fd[0] == primary_key_set:
        map["_".join(list(fd[0]))]=None
        #tables_of_2nf["_".join(list(fd[0]))]=set()

    for vfd in violated_fds:
      for every_val_in_rhs in vfd[1]:
        if every_val_in_rhs not in map:
          map[every_val_in_rhs]="_".join(vfd[0])

    for cfd in correct_fds:
      for every_val_in_rhs in cfd[1]:
        map[every_val_in_rhs]="_".join(cfd[0])



    for vfd in violated_fds:
      if '_'.join(vfd[0]) not in map:
        map['_'.join(vfd[0])]=None
      non_primes_in_lhs = []
      for attr in vfd[0]:
        if attr not in primary_key_set:
          non_primes_in_lhs.append(attr)
      for ever_nonprime_attr in non_primes_in_lhs:
        map[ever_nonprime_attr] = "_".join(vfd[0])

    #print("========== map ===========")
    #print(map)
    #print(data[0].keys())

    global_map = map

    #print(tables_of_2nf)
    for correct_fd in correct_fds:
      #[({'Professor'}, {'ProfessorEmail'})]
      rhs = correct_fd[1]
      #get the table of lhs
      set_iterator = iter(rhs)
      # Get the first element from the set
      first_element = next(set_iterator)
      #print(correct_fds)
      #print("========== first_element ===========")
      #print(first_element)

      while map[first_element] is not None:
        first_element = map[first_element]

      temp_key = first_element
      #print("========== temp_key ===========")
      #print(temp_key)

      #tables_of_2nf[temp_key] =
      for item in rhs:
        tables_of_2nf[temp_key].add(item)
      table_fds[temp_key].append(correct_fd)

      #print("========== tables_of_2nf ===========")
      #print(tables_of_2nf)


    tables_of_2nf_dfs={}

    original_df = pd.DataFrame(data)

    for table_name,corresponding_cols in tables_of_2nf.items():
      tables_of_2nf_dfs[table_name] = original_df[corresponding_cols]

    ##print("\n\n\n")
    #for table_name,df in tables_of_2nf_dfs.items():

      #print(table_name)
      #print("\n")
      #print(df)

    #print("===== table_fds ====")
    #print(table_fds)


    ###sending table_fds,tables_of_2nf_dfs for 3NF verification
    return table_fds,tables_of_2nf_dfs



    #remaining_attrs = set(data[0].keys()).difference(set().union(*decomposed_relations))
    #decomposed_relations.append(primary_key_set.union(remaining_attrs))

    #return decomposed_relations, remaining_fds

def is_in_1NF(data):
    # Using a common delimiter ';'
    # Modify based on what you think might be used to store multiple values
    delimiter = ','
    multi_value_attributes = set()
    for row in data:
        for attribute, value in row.items():
            if delimiter in value:
                print(f"Attribute {attribute} contains non-atomic values.")
                multi_value_attributes.add(attribute)
    return multi_value_attributes




def is_in_2NF(fds,key):
  "[({'StudentID'}, {'FirstName', 'LastName'})]"
  #print("fds : ",fds)
  #print("key : ",key)
  violated_fds = []
  correct_fds=[]
  for fd in fds:
    #print("fd1 : ",fd[1])
    condition1 = len(fd[0]) < len(key) and fd[0].issubset(key)
    if(condition1):
      violated_fds.append(fd)
      continue
    count = len(key)
    actualCount = 0
    for item in fd[0]:
      if item in key:
        actualCount += 1
    condition2 =  count !=   actualCount and actualCount>=1
    if(condition2):
      correct_fds.append(fd)
      continue

    #checking if lhs completely conisst of np attributes, declaring it as a safe fd
    flag = 0
    for item in fd[0]:
      if item in key:
        flag = 1
        break
    if flag == 0 or fd[0] == key:
      correct_fds.append(fd)




  #print("========= violated_fds =========")
  #print(violated_fds)
  return violated_fds,correct_fds







def is_in_3NF_internal(dependencies, headerList, candidateKey,faulty_dependency_indexes,key):
    #print("===== 3NF dependencies =========")
    #print(dependencies)
    transitive_dependencies = []
    #print("====== global_map ==========")
    #print(global_map)
    #print(candidateKey)
    i=0
    for fd in dependencies:
      #print("====== fd ==========")
      #print(fd)
      #====== fd ==========
      #({'Course'}, {'CourseEnd', 'Professor', 'CourseStart'})
      #====== fd ==========
      #({'Professor'}, {'ProfessorEmail'})
      lhs_attr = next(iter(fd[0]))
      for ever_right in fd[1]:
        if ever_right in key:
          continue
        #ever_right = professoremail
        A = global_map[ever_right] #professor #course
        if A is None:
          continue
        B = global_map[A] #course None
        if B is None:
          continue
        if global_map[A] == B:
          transitive_dependencies.append(({lhs_attr},{ever_right}))
          faulty_dependency_indexes.setdefault(candidateKey, []).append(i)


      i=i+1


    return transitive_dependencies,faulty_dependency_indexes









def main():
    # Reading CSV
    data = read_csv_file('Testing-BCNF-2.csv')
    #print("============ data =============")
    ##print(data)

    # Parsing Functional Dependencies
    fds = parse_functional_dependencies()
    #print("============ fds =============")
    #print(fds)

    # Determine data types
    #data_types = determine_data_types(data)

    # User Input for Normalization Choice
    #target_nf_choice = input("Choice of the highest normal form to reach (1: 1NF, 2: 2NF, 3: 3NF, B: BCNF, 4: 4NF, 5: 5NF): ")

    # Determine current normal form
    key = input("Key (can be composite): ").split(',')
    #print("=============key=================")
    #print(key)

    # User Input for Normalization Choice
    target_nf_choice = input("Choice of the highest normal form to reach (1: 1NF, 2: 2NF, 3: 3NF, B: BCNF, 4: 4NF, 5: 5NF): ")
    if target_nf_choice == '4':
      mvds = find_multivalued_dependencies()
      find_normal_form(data, fds, key,target_nf_choice,mvds)
    else:
      mvds = []
      find_normal_form(data, fds, key,target_nf_choice,mvds)

    if target_nf_choice == '5':
      # Decompose the table into 5NF
      decomposed_tables = decompose_to_5nf(data, fds)
      print("5NF form of table : ")
      decomposed_tables = [item for sublist in decomposed_tables for item in sublist]
      print(pd.DataFrame(data)[decomposed_tables])

# The result will be a list of decomposed tables, each in 5NF.


    # Output the current normal form
    #print(f"The current normal form of the input table is: {current_nf}")
    #tables = [set(data[0].keys())]
    #print("============ tables =============")
    #print(tables)

def find_multivalued_dependencies():
    mvds = []
    while True:
        mvd_input = input("Input Multi-Valued Dependencies (or 'exit'): ")
        if mvd_input == "exit":
            break
        mvd = mvd_input.split('->>')
        mvds.append((set(x.strip() for x in mvd[0].split(',')), set(x.strip() for x in mvd[1].split(','))))
    return mvds

if __name__ == "__main__":
    main()


Input Functional Dependencies (or 'exit'): Teacher,Subject -> Student
Input Functional Dependencies (or 'exit'): exit
Key (can be composite): Teacher,Subject
Choice of the highest normal form to reach (1: 1NF, 2: 2NF, 3: 3NF, B: BCNF, 4: 4NF, 5: 5NF): B
in 1NF
checking for 2NF...
Table is in 2NF
table Teacher_Subject is in 3NF
============== Table is in BCNF ===========
